### Imports and Configuration

In [1]:
import copy
import os
import statistics
from functools import reduce
from copy import deepcopy

import matplotlib.pyplot as plt
import numpy as np

if os.environ.get('DISPLAY') is not None:
    %matplotlib tk

import msmfcode_src
from msmfcode.core.logging import log
from msmfcode.core.config import *
from msmfcode.execution.parallel import ParallelExecutor, ParallelEvaluationExecutor
from msmfcode.models.cann import DMSMF, FMSMF, Grid, SSSF
from msmfcode.evaluation.plot import plot_weight_distribution, plot_neuron_activity_fixed_attractors, plot_neuron_activity_variable_attractors, plot_neuron_activity_static, save_plot, plot_distribution
from msmfcode.evaluation.algorithms import calc_kls_sizes, calc_kls_locs

# Configuration
PLOT_FILE_TYPE = 'pdf'
log.handlers[LogHandler.STREAM].setLevel(logging.DETAIL)
log.handlers[LogHandler.FILE].setLevel(logging.DETAIL)

### Evaluation - Single Run


In [2]:
experiment_id = 'single'

pe = ParallelExecutor(experiment_id, FMSMF)
pe.run()


# pe_ = deepcopy(pe)
#
# for i in range(len(pe.cans)):
#     pe.cans[i].p.I = 0
#     # pe.cans[i].p.J1 = 0
#     # pe.cans[i].p.field_ratio_threshold = 0.99
#
# pe.run(reset_fields=False, reset_weights=True)

# num_fields = np.sum(pe.cans[0].field_sizes > 0)
# avg_field_size = np.sum(pe.cans[0].field_sizes) / num_fields
# print(f'Avg field size: {avg_field_size}')



# pe.cans[0].plot(pos_range=range(100, 160))

# plot_neuron_activity_static(pe.cans[0], neuron_range=[0,1,2,3,4])


> Starting experiment single-27: eval_2-1-b-01
>> Starting evaluation run
>> Starting batch run [1/2]
>> Started all CAN simulations
>> Finished all CAN simulations
>> Starting batch run [2/2]
>> Started all CAN simulations
>> Finished all CAN simulations
-------------------------------------------------------------------------------------
>> Metrics after 20 runs:
>> pos_error_mean_mean = 0.16
>> pos_error_mean_min = 0.13
>> pos_error_mean_max = 0.19
>> pos_error_mean_std = 0.01
>> pos_error_mean_median = 0.17
>> pos_error_mean_q1 = 0.15
>> pos_error_mean_q3 = 0.17
>> pos_error_mean_whishi = 0.19
>> pos_error_mean_whislo = 0.13
>> pos_error_std_mean = 0.25
>> pos_error_std_min = 0.23
>> pos_error_std_max = 0.28
>> pos_error_std_std = 0.01
>> pos_error_std_median = 0.25
>> pos_error_std_q1 = 0.25
>> pos_error_std_q3 = 0.26
>> pos_error_std_whishi = 0.28
>> pos_error_std_whislo = 0.23
>> pos_error_min_mean = 0.00
>> pos_error_max_mean = 1.14
>> pos_error_num_cat_mean = 0.00
>> pos_error

In [13]:
print()

pe.cans[0].plot(pos_range=range(180, 280))

In [15]:
plot_neuron_activity_fixed_attractors(pe.cans[0], pause_time=3.0)

KeyboardInterrupt: 

In [64]:
print(f'Mean: {np.mean(pe.cans[0].W)}')
print(f'Min: {np.min(pe.cans[0].W)}')
print(f'Max: {np.max(pe.cans[0].W)}')
print(f'Nw>0: {np.count_nonzero(pe.cans[0].W)}')

a = plt.hist(pe.cans[0].W.flatten())


Mean: -0.5193361050091689
Min: -1.4790213944420878
Max: 0.0
Nw>0: 2450


In [66]:
print(f'Mean: {np.mean(pe.cans[0].W)}')
print(f'Min: {np.min(pe.cans[0].W)}')
print(f'Max: {np.max(pe.cans[0].W)}')
print(f'Nw>0: {np.count_nonzero(pe.cans[0].W)}')

a = plt.hist(pe.cans[0].W.flatten())

Mean: -0.24637800045999966
Min: -0.8311357933607728
Max: 0.004999999999999998
Nw>0: 2450


In [3]:
# pe.experiment_num = 11
pe.save_data(save_plot=False, save_model=True)

In [ ]:
# np.mean(self.m) * self.p.num_neurons * self.p.env_length

print(np.mean(pe.cans[0].m[0,:]))
print(np.mean(pea.cans[0].m[0,:]))

print(pe.cans[0].p.num_neurons_per_module)
print(pea.cans[0].p.num_neurons_per_module)

### Find id of best/worst performing network

In [8]:
# Code
pos_error_mean = pe.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values
net_ids_sorted = np.argsort(pos_error_mean)

print(f'Best ID = {net_ids_sorted[0]}')
print(f'Worst ID = {net_ids_sorted[-1]}')

best_net_id = net_ids_sorted[0]
worst_net_id = net_ids_sorted[-1]

Best ID = 2
Worst ID = 7


### Plot: Metric values for all CANs

In [ ]:
#
# Plot mean error distribution
#
import matplotlib.pyplot as plt

metric = Metric.POS_ERROR_MEAN
global_metric = 'mean'
metric_name = f'{metric}_{global_metric}'

plt.hist(pe.eval_data.eval_metrics[metric].values, bins=40)
print(f'Mean: {pe.eval_data.eval_metrics[metric].mean}')
print(f'Median: {pe.eval_data.eval_metrics[metric].median}')

### Plot: Fields of CAN

In [3]:
# plot_neuron_activity_static(pe.cans[0], neuron_range=range(5), save_plot=True, fig_type_name='3-1-b-s-03a')
# plot_neuron_activity_static(pe_dead.cans[0], neuron_range=range(5, 10))
plot_neuron_activity_static(pe.cans[0], neuron_range=range(10))


In [12]:
pe.cans[7].plot(pos_range=range(100, 160))

### Perform subsequent experiment with lesions

In [ ]:
experiment_id = 'single'

pe_dead = copy.deepcopy(pe)

pe_dead.init_eval_data()

# pe = ParallelExecutor(experiment_id, DMSMF)
for i_can in range(len(pe_dead.cans)):
    pe_dead.cans[i_can].p.prob_dead_neurons = 0.3
    pe_dead.cans[i_can].set_dead_neurons()
pe_dead.run(reset_fields=False, reset_weights=False)


### Plot: Field size distribution

In [ ]:
metric = Metric.AVG_NUM_FIELDS_PER_NEURON

print(pe.eval_data.eval_metrics[metric].values[2])
print(pe.eval_data.eval_metrics[metric].values[11])

plot_distribution(pe.cans[2].field_sizes, nbins=np.arange(0, 10, step=0.05), y_lim=(0, 30))
plt.figure()
plot_distribution(pe.cans[11].field_sizes, nbins=np.arange(0, 10, step=0.05), y_lim=(0, 30))


#### Evaluation of field location distribution

In [ ]:
# Config
n_bins = 50
use_sorted_ids = True

# Code
pos_error_mean = pe.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values
pos_error_std = pe.eval_data.eval_metrics[Metric.POS_ERROR_STD].values
net_ids_sorted = np.argsort(pos_error_mean)

if use_sorted_ids:
    net_id_range = net_ids_sorted
else:
    net_id_range = range(len(pe.cans))

# net_id_range = [1]

fig_kls, axs_kls = plt.subplots(nrows=4, ncols=4)
fig_kls.tight_layout()

fig_loc, axs_loc = plt.subplots(nrows=4, ncols=4)
fig_loc.tight_layout()

for i, i_net in enumerate(net_id_range):
    # kls = get_prob_bins(pe.cans[i_net], 0.66, 1.8, num_bins=n_bins)
    kls = calc_kls_locs(pe.cans[i_net], 0.66, pe.cans[i_net].p.env_length, num_bins=n_bins, lower_ppf=0.0, upper_ppf=1.0)
    print(f'Network [{i_net}]: mean-kl = {np.mean(kls)}, std-kl = {np.std(kls)}, pos-error-mean = {pos_error_mean[i_net]}')

    axs_x = int(i / 4)
    axs_y = int(i % 4)
    title = f'Net-{i_net}: kl-mean/std={np.mean(kls):.3f}/{np.std(kls):.3f},\npos-error-mean={pos_error_mean[i_net]:.3f},\npos-error-std={pos_error_std[i_net]:.3f}'

    axs_kls[axs_x, axs_y].hist(kls, bins=10)
    axs_kls[axs_x, axs_y].set_title(title)

    # plot_distribution(pe.cans[i_net].field_locs, ax=axs[axs_x, axs_y], title=title, nbins=np.arange(0, 10, step=0.05))
    plot_distribution(pe.cans[i_net].field_locs, ax=axs_loc[axs_x, axs_y], title=title, nbins=n_bins)


#### Evaluation of KL-D Mean/Std for field sizes and locations

In [24]:
#
# 2-2: Distribution of field locations and sizes from original distributions
#

## Config
evalutated_prop = 'locations'
n_bins = 20

## Code
# fig, axs = plt.subplots(nrows=2, constrained_layout=True, figsize=(10, 5))
fig, axs = plt.subplots(nrows=2, figsize=(10, 5))

fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.ylabel('KL-Divergence (mean/std)', fontsize='12', labelpad=44)
plt.yticks([])

if evalutated_prop == 'sizes':
    fig.suptitle('KL-Divergence - Field size distribution vs. gamma distribution', fontsize='14', weight='bold')
elif evalutated_prop == 'locations':
    fig.suptitle('KL-Divergence - Field location distribution vs. uniform distribution', fontsize='14', weight='bold')

kld_means = list()
kld_stds = list()

pos_error_mean = pe_many.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values
net_ids_sorted = np.argsort(pos_error_mean)

for i, i_net in enumerate(net_ids_sorted):
    if evalutated_prop == 'sizes':
        kls = calc_kls_sizes(pe_many.cans[i_net], 0.66, 1.8, num_bins=n_bins)
    elif evalutated_prop == 'locations':
        kls = calc_kls_locs(pe_many.cans[i_net], 0.66, pe_many.cans[i_net].p.env_length, num_bins=n_bins, lower_ppf=0.0, upper_ppf=1.0)
    # print(f'Network [{i_net}]: mean-kl = {np.mean(kls)}, std-kl = {np.std(kls)}, pos-error-mean = {pos_error_mean[i_net]}')

    kld_means.append(np.mean(kls))
    kld_stds.append(np.std(kls))

x = np.array(range(len(kld_means)))

axs[0].errorbar(x, kld_means, kld_stds, fmt='o')
xtick_labels = [f'{i:.2f}' for i in np.sort(pos_error_mean)]
axs[0].set_xticks(x, xtick_labels)
axs[0].set_xlabel('Mean Positional Error (m)', fontsize='12', labelpad=12)
axs[0].set_title('Model A')

# axs[0].set_ylabel('KL-Divergence (Mean/Std)', fontsize='12', labelpad=12)


kld_means = list()
kld_stds = list()

pos_error_mean = pe_few.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values
net_ids_sorted = np.argsort(pos_error_mean)

for i, i_net in enumerate(net_ids_sorted):
    if evalutated_prop == 'sizes':
        kls = calc_kls_sizes(pe_few.cans[i_net], 0.66, 1.8, num_bins=n_bins)
    elif evalutated_prop == 'locations':
        kls = calc_kls_locs(pe_few.cans[i_net], 0.66, pe_few.cans[i_net].p.env_length, num_bins=n_bins, lower_ppf=0.0, upper_ppf=1.0)
    # print(f'Network [{i_net}]: mean-kl = {np.mean(kls)}, std-kl = {np.std(kls)}, pos-error-mean = {pos_error_mean[i_net]}')

    kld_means.append(np.mean(kls))
    kld_stds.append(np.std(kls))

# ax2 = axs.twiny()

axs[1].errorbar(x+0.5, kld_means, kld_stds, fmt='o', color='C1')
xtick_labels = [f'{i:.2f}' for i in np.sort(pos_error_mean)]
axs[1].set_xticks(x+0.5, xtick_labels)
axs[1].set_xlabel('Mean Positional Error (m)', fontsize='12', labelpad=12)
axs[1].set_title('Model B')
# axs[1].grid(linestyle='--')

# plt.tight_layout()

fig.subplots_adjust(left=0.09, bottom=0.12, right=0.99, top=0.843, hspace=0.9)


experiment_description = f'2-2_kld-field-{evalutated_prop}'
fig_name = f'{DMSMF.__name__}_experiment_{experiment_description}.pdf'
file_path = join(EXPERIMENT_FOLDERS[DMSMF.__name__], EXPERIMENT_SUBFOLDERS[FileType.FIGURE], fig_name)

fig.savefig(file_path)



### Evaluation of field location vs. bin location

In [18]:
def get_field_loc_dist(net):
    field_locs = (net.field_locs + 0.5 * net.p.disc_step) / net.p.disc_step
    field_locs = np.where(field_locs % 1 > 0.5, 1 - field_locs % 1, field_locs % 1)
    field_locs /= 2
    field_locs[net.field_sizes == 0] = 0

    return field_locs[field_locs > 0].flatten()


pos_error_mean = pe.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values

net_id_min = np.argmin(pos_error_mean)
net_id_max = np.argmax(pos_error_mean)

min_dists = get_field_loc_dist(pe.cans[net_id_min])
max_dists = get_field_loc_dist(pe.cans[net_id_max])

plt.figure(figsize=(12,7))
plt.subplot(1, 2, 1)
plt.violinplot(min_dists, showmeans=False, showextrema=True, showmedians=True)
plt.title(f'Minimum error = {pos_error_mean[net_id_min]}')
plt.subplot(1, 2, 2)
plt.violinplot(max_dists, showmeans=False, showextrema=True, showmedians=True)
plt.title(f'Maximum error = {pos_error_mean[net_id_max]}')


print(f'Median [min] = {np.median(min_dists)}')
print(f'Median [max] = {np.median(max_dists)}')

print(f'Mean [min] = {np.mean(min_dists)}')
print(f'Mean [max] = {np.mean(max_dists)}')


Median [min] = 0.1264547677291432
Median [max] = 0.1299440764851667
Mean [min] = 0.12703949117053975
Mean [max] = 0.12751071112333895


In [23]:
fig, axs = plt.subplots(nrows=2, figsize=(10, 5))

fig.suptitle('Field-bin-center distance', fontsize='14', weight='bold')

fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.ylabel('Median distance (m)', fontsize='12', labelpad=44)
plt.yticks([])

## Calculate dist for many
pos_error_mean = pe_many.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values
sorted_ids = np.argsort(pos_error_mean)

x = np.array(range(len(pos_error_mean)))

median_bin_dists_many = {Statistics.MEAN: [], Statistics.MEDIAN: [], Statistics.STD: []}
for i_net in sorted_ids:
    min_dists_many = get_field_loc_dist(pe_many.cans[i_net])
    # median_bin_dists_many.append(np.median(min_dists_many))
    for stat in median_bin_dists_many.keys():
        median_bin_dists_many[stat].append(getattr(np, stat)(min_dists_many))

# axs[0].plot(median_bin_dists_many)
axs[0].errorbar(x+0.5, median_bin_dists_many[Statistics.MEAN], median_bin_dists_many[Statistics.STD], fmt='o', color='C0')
xtick_labels = [f'{i:.2f}' for i in np.sort(pos_error_mean)]
axs[0].set_xticks(x, xtick_labels)
axs[0].set_xlabel('Mean Positional Error of Network (m)', fontsize='12', labelpad=12)
axs[0].set_title('Model A')

## Same for few
pos_error_mean = pe_few.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values
sorted_ids = np.argsort(pos_error_mean)

median_bin_dists_few = {Statistics.MEAN: [], Statistics.MEDIAN: [], Statistics.STD: []}
for i_net in sorted_ids:
    min_dists_few = get_field_loc_dist(pe_few.cans[i_net])
    for stat in median_bin_dists_few.keys():
        median_bin_dists_few[stat].append(getattr(np, stat)(min_dists_few))

# axs[1].plot(median_bin_dists_few)
axs[1].errorbar(x+0.5, median_bin_dists_few[Statistics.MEAN], median_bin_dists_few[Statistics.STD], fmt='o', color='C1')
xtick_labels = [f'{i:.2f}' for i in np.sort(pos_error_mean)]
axs[1].set_xticks(x, xtick_labels)
axs[1].set_xlabel('Mean Positional Error of Network (m)', fontsize='12', labelpad=12)
axs[1].set_title('Model B')

# Adjust plot size
fig.subplots_adjust(left=0.09, bottom=0.12, right=0.99, top=0.843, hspace=0.9)

# Save plot
experiment_description = f'2-2_field-bin-dist'
fig_name = f'{DMSMF.__name__}_experiment_{experiment_description}.pdf'
file_path = join(EXPERIMENT_FOLDERS[DMSMF.__name__], EXPERIMENT_SUBFOLDERS[FileType.FIGURE], fig_name)

fig.savefig(file_path)

# plt.title(f'Median distance between field locations and bin centers')


### Evaluation of false positives/negatives

In [78]:
pos_error_mean = pe.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values
sorted_ids = np.argsort(pos_error_mean)

net_id_min = np.argmin(pos_error_mean)
net_id_max = np.argmax(pos_error_mean)

fig = plt.figure(1)

ax = fig.add_subplot(111)
ax.plot(np.take(pe.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values, sorted_ids), 'x')

ax2 = ax.twinx()
ax2.plot(np.take(pe.eval_data.eval_metrics[Metric.ACTIVITY_FALSE_POSITIVES_NUM].values, sorted_ids), 'x', color='C1')
ax2.plot(np.take(pe.eval_data.eval_metrics[Metric.ACTIVITY_FALSE_NEGATIVES_NUM].values, sorted_ids), 'x', color='C2')



In [22]:
fig, axs = plt.subplots(nrows=2, figsize=(12, 8))

fig.suptitle('False positive/negative count per network', fontsize='14', weight='bold')

fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.ylabel('Median distance (m)', fontsize='12', labelpad=44)
plt.yticks([])

## Calculate dist for many
pos_error_mean = pe_many.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values
sorted_ids = np.argsort(pos_error_mean)

x = np.array(range(len(pos_error_mean)))

# axs[0].plot(np.take(pe_many.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values, sorted_ids), 'x')
#
# ax2 = axs[0].twinx()
# ax2.plot(np.take(pe_many.eval_data.eval_metrics[Metric.ACTIVITY_FALSE_POSITIVES_NUM].values, sorted_ids), 'x', color='C1')
# ax2.plot(np.take(pe_many.eval_data.eval_metrics[Metric.ACTIVITY_FALSE_NEGATIVES_NUM].values, sorted_ids), 'x', color='C2')
axs[0].scatter(x, np.take(pe_many.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values, sorted_ids), facecolors='None', edgecolors=f'C0')

ax2 = axs[0].twinx()
ax2.scatter(x, np.take(pe_many.eval_data.eval_metrics[Metric.ACTIVITY_FALSE_POSITIVES_NUM].values, sorted_ids), facecolors='None', edgecolors=f'C1')
ax2.scatter(x, np.take(pe_many.eval_data.eval_metrics[Metric.ACTIVITY_FALSE_NEGATIVES_NUM].values, sorted_ids), facecolors='None', edgecolors=f'C2')

xtick_labels = [f'{i:.2f}' for i in np.sort(pos_error_mean)]
axs[0].set_xticks(x, xtick_labels)
axs[0].set_xlabel('Mean Positional Error of Network (m)', fontsize='12', labelpad=12)
axs[0].set_title('Model A')

## Same for few
pos_error_mean = pe_few.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values
sorted_ids = np.argsort(pos_error_mean)

axs[1].scatter(x, np.take(pe_few.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values, sorted_ids), facecolors='None', edgecolors=f'C0')

ax3 = axs[1].twinx()
ax3.scatter(x, np.take(pe_few.eval_data.eval_metrics[Metric.ACTIVITY_FALSE_POSITIVES_NUM].values, sorted_ids), facecolors='None', edgecolors=f'C1')
ax3.scatter(x, np.take(pe_few.eval_data.eval_metrics[Metric.ACTIVITY_FALSE_NEGATIVES_NUM].values, sorted_ids), facecolors='None', edgecolors=f'C2')

xtick_labels = [f'{i:.2f}' for i in np.sort(pos_error_mean)]
axs[1].set_xticks(x, xtick_labels)
axs[1].set_xlabel('Mean Positional Error of Network (m)', fontsize='12', labelpad=12)
axs[1].set_title('Model B')

# Adjust plot size
fig.subplots_adjust(left=0.09, bottom=0.12, right=0.99, top=0.843, hspace=0.9)

# Save plot
experiment_description = f'2-2_false-pos-neg'
fig_name = f'{DMSMF.__name__}_experiment_{experiment_description}.pdf'
file_path = join(EXPERIMENT_FOLDERS[DMSMF.__name__], EXPERIMENT_SUBFOLDERS[FileType.FIGURE], fig_name)

fig.savefig(file_path)

### Evaluation of perc unique field combinations

In [21]:
pe = pe_few

pos_error_mean = pe.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values
sorted_ids = np.argsort(pos_error_mean)
print(np.sort(pos_error_mean))

for k in np.arange(0.001, 0.01, 0.001):
    num_combs = []
    for i in sorted_ids:

        net = pe.cans[i]

        arr = net.m > k

        num_unique_field_combinations = len(np.unique(reduce(lambda a,b: 2*a+b, arr)))
        perc_unique_field_combinations = num_unique_field_combinations / net.num_bins

        num_combs.append(perc_unique_field_combinations)

    plt.plot(num_combs)
plt.show()


[0.9520202  0.98579545 0.99368687 1.02714646 1.03598485 1.03977273
 1.14267677 1.15340909 1.43813131 1.51641414 1.5719697  1.62058081
 1.67424242 1.73989899 1.76010101 1.83143939 1.84090909 2.06060606
 2.46275253 2.69255051]


### 3-3: Evaluation of lateral connection benefits in MSMF models

In [15]:
plot_neuron_activity_static(pe_.cans[0], neuron_range=range(5), save_plot=True, experiment_num=18, fig_type_name='3-3-a')
plot_neuron_activity_static(pe.cans[0], neuron_range=range(5), save_plot=True, experiment_num=4, fig_type_name='3-3-a')

### Evaluation - Multi Run - MSMF Fixed

In [ ]:
experiment_id = 'c'

pe = ParallelEvaluationExecutor(experiment_id, DMSMF)

pe.run()
pe.save_data(show_plots=False, save_plots=True)

### Object (De-)Serialization

In [ ]:
# Object Serialization
from msmfcode.evaluation.data import save_model_object

save_model_object(pe, DMSMF, model_description='2-2_large-pos-std_many-fields', model_id='opt-05-167-08')

In [13]:
# Object Deserialization
from msmfcode.evaluation.data import load_model_object

# pe_nlat = load_model_object(ParallelExecutor, DMSMF, model_description='experiment', model_id='single-14')
pe = load_model_object(ParallelExecutor, DMSMF, model_description='experiment', model_id='single-18')


# pe = load_model_object(ParallelExecutor, DMSMF, model_description='pos-error-zero', model_id='gen-0-ent-7')


# Backup

In [ ]:
for mean_error_pe, mean_error_pe_dead, net_id in (zip(pe.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values, pe_dead.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values, range(len(pe.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values)))):
    print(f'CAN [{net_id}]: Mean Pos Error = {mean_error_pe:.3f}  |  {mean_error_pe_dead:.3f}  ||  {mean_error_pe_dead-mean_error_pe:.3f}')

In [ ]:


# bins = list(np.arange(0.25, 6, 0.25))
np.set_printoptions(suppress=True)

# g = gamma(a=0.66, scale=1.8, loc=0)
# lower_bound = g.ppf(0.01)
# upper_bound = g.ppf(0.99)
# N = 25
# bins, step = np.linspace(lower_bound, upper_bound, N, retstep=True)
#
# bin_prob_neuron = get_prob_bins(pe.cans[10], bins)
#
#
# bin_prob_gamma = np.zeros(len(bins))
#
# for i_bin, bin in enumerate(bins):
#     if i_bin == 0:
#         value_a = g.pdf(0.000000001)
#     else:
#         value_a = g.pdf(bins[i_bin-1])
#     value_b = g.pdf(bin)
#     avg = np.mean([value_a, value_b])
#     bin_prob_gamma[i_bin] = avg
#
# bin_prob_gamma /= np.sum(bin_prob_gamma)


# print(bin_prob_gamma)

# Config
n_bins = 40

# Code
pos_error_mean = pe.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values

net_id_min = np.argmin(pos_error_mean)
net_id_max = np.argmax(pos_error_mean)

# net_id_min = 15
# net_id_max = 10

print(f'Calculating KL for minimum error net (mean-pos-error = {pos_error_mean[net_id_min]})')
# kls_a = calc_kls_sizes(pe.cans[net_id_min], 0.66, 1.8, num_bins=20)
kls_a = calc_kls_locs(pe.cans[net_id_min], 0.66, pe.cans[net_id_min].p.env_length, num_bins=50, lower_ppf=0.0, upper_ppf=1.0)
print(f'Network [{net_id_min}]: mean-kl = {np.mean(kls_a)}, std-kl = {np.std(kls_a)}, pos-error-mean = {pos_error_mean[net_id_min]}')

print(f'\nCalculating KL for maximum error net (mean-pos-error = {pos_error_mean[net_id_max]})')
# kls_b = calc_kls_sizes(pe.cans[net_id_max], 0.66, 1.8, num_bins=20)
kls_b = calc_kls_locs(pe.cans[net_id_max], 0.66, pe.cans[net_id_max].p.env_length, num_bins=50, lower_ppf=0.0, upper_ppf=1.0)
print(f'Network [{net_id_max}]: mean-kl = {np.mean(kls_b)}, std-kl = {np.std(kls_b)}, pos-error-mean = {pos_error_mean[net_id_max]}')

fig, axs = plt.subplots(nrows=2, ncols=1)
fig.tight_layout()
axs[0].hist(kls_a, bins=n_bins)
axs[0].set_title(f'Net-{net_id_min} with minimal error = {pos_error_mean[net_id_min]:3f}')
axs[1].hist(kls_b, bins=n_bins)
axs[1].set_title(f'Net-{net_id_max} with maximal error = {pos_error_mean[net_id_max]:3f}')


In [ ]:
## Plot KLs for all networks

# Config
n_bins = 20
use_sorted_ids = True

# Code
pos_error_mean = pe.eval_data.eval_metrics[Metric.POS_ERROR_MEAN].values
pos_error_std = pe.eval_data.eval_metrics[Metric.POS_ERROR_STD].values
net_ids_sorted = np.argsort(pos_error_mean)

if use_sorted_ids:
    net_id_range = net_ids_sorted
else:
    net_id_range = range(len(pe.cans))

# net_id_min = np.argmin(pos_error_mean)
# net_id_max = np.argmax(pos_error_mean)
# net_id_range = [net_id_min, net_id_max]

# net_id_range = [1]

fig, axs = plt.subplots(nrows=4, ncols=4)
fig.tight_layout()

for i, i_net in enumerate(net_id_range):
    kls = calc_kls_sizes(pe.cans[i_net], 0.66, 1.8, num_bins=n_bins)
    # kls = calc_kls_locs(pe.cans[i_net], 0.66, pe.cans[i_net].p.env_length, num_bins=n_bins, lower_ppf=0.0, upper_ppf=1.0)
    print(f'Network [{i_net}]: mean-kl = {np.mean(kls)}, std-kl = {np.std(kls)}, pos-error-mean = {pos_error_mean[i_net]}')

    axs_x = int(i / 4)
    axs_y = int(i % 4)
    axs[axs_x, axs_y].hist(kls, bins=n_bins)
    axs[axs_x, axs_y].set_title(f'Net-{i_net}: kl-mean/std={np.mean(kls):.3f}/{np.std(kls):.3f},\npos-error-mean={pos_error_mean[i_net]:.3f},\npos-error-std={pos_error_std[i_net]:.3f}')
